In [1]:
### importing all the libraries needed
import time
from time import sleep
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd

### Shopee Malaysia

In [2]:
%%time
### initial set up of selenium and chrome settings. Selenium is imitating as a Chrome broswer. 
### credits @ https://stackoverflow.com/questions/62057645/how-to-scrape-data-from-shopee-using-beautiful-soup

driver = webdriver.Chrome(ChromeDriverManager().install()) #this ensures you are using the latest chrome version everytime! Dont even need to find driver_path.  
chrome_options = Options()
chrome_options.headless = True
chrome_options.add_argument("--window-size=1920,1200")
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument('user-data-dir=C:\\Users\\username\\AppData\\Local\\Google\\Chrome\\User Data\\Default')

# To disable the message, "Chrome is being controlled by automated test software"
chrome_options.add_argument("disable-infobars")

# Pass the argument 1 to allow and 2 to block
chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 2})

### get the browser load the url and display stuffs 
driver.get("https://shopee.com.my/shocking_sale")
driver.find_element_by_xpath('//button[text()="English"]').click()

links = [link.get_attribute('href') for link in driver.find_elements_by_class_name('flash-sale-item-card-link')]

web_list = []
sold_out_list = []

for link in links:
    driver.get(link)
    sleep(30) # seconds. it is v impt to give browser some time to sleep, so that the scrolling can be completed and get full HTML
    selenium_html = driver.page_source
    soup = BeautifulSoup(selenium_html, 'lxml') #lxml parsing is faster than html parsing 
    if soup.find_all('div', class_ = '_3YfrdB'): # if product is sold out
        link1 = driver.find_element_by_class_name('link-to-keep-parent-style')
        link2 = link1.get_attribute('href')
        driver.get(link2)
        selenium_html = driver.page_source
        soup = BeautifulSoup(selenium_html, 'lxml') #lxml parsing is faster than html parsing 
        cat_so = [x.get_text() for x in soup.find_all(class_ = "_3QRNmL")]        
        sold_out_list.append(cat_so)
        my_sold_out_df = pd.DataFrame(sold_out_list)
        my_sold_out_df = my_sold_out_df.iloc[: , :4]
        my_sold_out_df.columns = ['Main Category','Sub Category','Product','Product Name']
    else: # if product is still in stock
        cat = [x.get_text() for x in soup.find_all('span', class_ = "_3QRNmL")]
        web_list.append(cat)
        my_in_stock_df = pd.DataFrame(web_list)
        my_in_stock_df = my_in_stock_df.iloc[: , :4]
        my_in_stock_df.columns = ['Main Category','Sub Category','Product','Product Name']
    
driver.quit() #to automatically close the browser



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\sherm\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


Wall time: 6min 14s


In [3]:
if len(sold_out_list) == 0:
    my_in_stock_df['Sold Out'] = 'No'
    my_df = my_in_stock_df 
else:
    my_sold_out_df['Sold Out'] = 'Yes'
    my_in_stock_df['Sold Out'] = 'No'
    my_df = my_sold_out_df.append(my_in_stock_df)

In [4]:
my_df

,Main Category,Sub Category,Product,Product Name,Sold Out
0,Men Shoes,Sneakers,Skechers Men Usa Delson 2 Shoes 210025-GRY,Men Shoes,No
1,Sports & Outdoors,Sports Footwear,Running Shoes,Skechers Women Performance GOrun Pulse Shoes -...,No
2,Women Shoes,Flat Sandals & Flip Flops,Flat Sandals,Skechers Women Cali Side Lines 2 Sandals - 873...,No
3,Audio,"Earphones, Headphones & Headsets",OPPO Enco Buds l 24Hr Battery Life l IP54 Wate...,Audio,No
4,Mobile & Gadgets,Mobile Phones,OPPO A15s Smartphone | 4GB RAM + 64GB ROM | AI...,Mobile & Gadgets,No
5,Mobile & Gadgets,Mobile Phones,OPPO A74 Smartphone l 6GB+128GB l 33W Flash Ch...,Mobile & Gadgets,No
6,Beauty,Bath & Body Care,Body Deodorants,NIVEA Female Deodorant Spray - Hokkaido Rose (...,No
7,Beauty,Skincare,Facial Serum & Essence,NIVEA Luminous630 Spotclear Booster Serum 30ml,No
8,Beauty,Skincare,Facial Serum & Essence,NIVEA Luminous630 Anti Dark Spot Gold Serum 30ml,No
9,Home Appliances,Small Household Appliances,Irons & Steamers,KONKA Portable Handheld Garment Steam Ironing ...,No


In [5]:
# saving excel file named according to datetime of extraction 
timestr = time.strftime("%Y%m%d-%H%M")
my_df.to_excel('Shopee_MY'+ timestr + '.xlsx')

### Shopee Singapore

In [6]:
%%time
### initial set up of selenium and chrome settings. Selenium is imitating as a Chrome broswer. 
### credits @ https://stackoverflow.com/questions/62057645/how-to-scrape-data-from-shopee-using-beautiful-soup

driver = webdriver.Chrome(ChromeDriverManager().install()) #this ensures you are using the latest chrome version everytime! Dont even need to find driver_path.  
chrome_options = Options()
chrome_options.headless = True
chrome_options.add_argument("--window-size=1920,1200")
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument('user-data-dir=C:\\Users\\username\\AppData\\Local\\Google\\Chrome\\User Data\\Default')

# To disable the message, "Chrome is being controlled by automated test software"
chrome_options.add_argument("disable-infobars")

# Pass the argument 1 to allow and 2 to block
chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 2})

### get the browser load the url and display stuffs 
driver.get("https://shopee.sg/flash_deals")

links = [link.get_attribute('href') for link in driver.find_elements_by_class_name('flash-sale-item-card-link')]

web_list = []
sold_out_list = []

for link in links:
    driver.get(link)
    sleep(30) # seconds. it is v impt to give browser some time to sleep, so that the scrolling can be completed and get full HTML
    selenium_html = driver.page_source
    soup = BeautifulSoup(selenium_html, 'lxml') #lxml parsing is faster than html parsing 
    if soup.find_all('div', class_ = '_3YfrdB'): # if product is sold out
        link1 = driver.find_element_by_class_name('link-to-keep-parent-style')
        link2 = link1.get_attribute('href')
        driver.get(link2)
        selenium_html = driver.page_source
        soup = BeautifulSoup(selenium_html, 'lxml') #lxml parsing is faster than html parsing 
        cat_so = [x.get_text() for x in soup.find_all(class_ = "_3QRNmL")]  
        sold_out_list.append(cat_so)
        sg_sold_out_df = pd.DataFrame(sold_out_list)
        sg_sold_out_df = sg_sold_out_df.iloc[: , :4]
        sg_sold_out_df.columns = ['Main Category','Sub Category','Product','Product Name']
    else: # if product is still in stock
        cat = [x.get_text() for x in soup.find_all('span', class_ = "_3QRNmL")]
        web_list.append(cat)
        sg_in_stock_df = pd.DataFrame(web_list)
        sg_in_stock_df= sg_in_stock_df.iloc[: , :4]
        sg_in_stock_df.columns = ['Main Category','Sub Category','Product','Product Name']
    
driver.quit() #to automatically close the browser



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\sherm\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


Wall time: 6min 15s


In [7]:
if len(sold_out_list) == 0:
    sg_in_stock_df['Sold Out'] = 'No'
    sg_df = sg_in_stock_df 
else:
    sg_sold_out_df['Sold Out'] = 'Yes'
    sg_in_stock_df['Sold Out'] = 'No'
    sg_df = sg_sold_out_df.append(sg_in_stock_df)

In [8]:
sg_df

,Main Category,Sub Category,Product,Product Name,Sold Out
0,Mobile & Gadgets,Mobile Phones,Samsung Galaxy S20 FE 5G 256GB,Mobile & Gadgets,No
1,Mobile & Gadgets,Tablets,"Samsung Galaxy Tab S6 Lite 10.4"" Wi-Fi",Mobile & Gadgets,No
2,Computers & Accessories,Laptops,"HUAWEI MateBook 14 Laptop | 14"" 2K Full View D...",Computers & Accessories,No
3,Stationery,Writing & Correction,Pens & Inks,NIKKI 1pc 0.5mm Multi-Color Press Gel Pen Retr...,No
4,Computers & Accessories,Laptops,HP Laptop 15s-fq2015TU / 11th Gen Intel i5-113...,Computers & Accessories,No
5,Men Shoes,Sneakers,adidas ORIGINALS I-5923 Shoes Women Blue Sneak...,Men Shoes,No
6,Audio,"Earphones, Headphones & Headsets",Baseus WM01 TWS Bluetooth Earphones Stereo Wir...,Audio,No
7,Computers & Accessories,Peripherals & Accessories,Laptop Chargers & Adaptors,Vention Active USB C Hub Converter 5 in 1 Adap...,No
8,Mobile & Gadgets,Accessories,"Cables, Chargers & Converters",【1 Year Warranty】UGREEN 65W UK Plug GaN Charge...,No
9,Food & Beverage,Alcoholic Beverages,Beer & Cider,Carlsberg Alcohol Free Pilsner Beer Can 330ml ...,No


In [9]:
# saving excel file named according to datetime of extraction 
timestr = time.strftime("%Y%m%d-%H%M")
sg_df.to_excel('Shopee_SG'+ timestr + '.xlsx')